In [1]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u



In [2]:
filename = "/home/gabriel/hdd4TB/photsat_catalogues/IPSL/photsat_IPSL.csv"
data = pd.read_csv(filename)

KeyboardInterrupt: 

In [ ]:
data.columns

Index(['sourceId', 'sourceIdExt', 'ref_epoch', 'RA', 'DEC', 'RA_error',
       'DEC_error', 'pmRA', 'pmDEC', 'pmRA_error', 'pmDEC_error', 'VIS1',
       'VIS2', 'UV', 'VIS1_error', 'VIS2_error', 'UV_error', 'ExtMag',
       'ExtCol', 'ExtMag_error', 'ExtCol_error', 'crowding', 'refMag',
       'refCol', 'Id'],
      dtype='object')

In [ ]:
ra_list = data[data["UV"]>0]['RA'].values
dec_list = data[data["UV"]>0]['DEC'].values

In [5]:
uv_data = len(data[data["UV"]>0])
uv_data_portion = uv_data/len(data)*100

print(f"DATA: {len(data)}")
print(f"UV DATA: {uv_data} - ({uv_data_portion})%")



DATA: 40107889
UV DATA: 2891785 - (7.2100154660346245)%


In [ ]:
from tqdm import tqdm

In [ ]:
fov = 8
h_pix=2048
v_pix=2048

min_distance=fov/2048/np.sqrt(2) *np.pi/180
block_size=1000
n_stars = len(ra_list)
proximity_threshold = min_distance*(1+5)

In [ ]:
coords = SkyCoord(ra=ra_list * u.deg, dec=dec_list * u.deg, frame='icrs')

In [ ]:
coords_radians = np.vstack([coords.ra.radian, coords.dec.radian]).T

In [ ]:
from sklearn.neighbors import BallTree

from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager
import time

In [11]:
tree = BallTree(coords_radians, metric='haversine')

: 

EJECUCIÓN DE BÚSQUEDA DE MÁS CERCANAS

In [12]:
# Función para estimar el tiempo

def find_nearest_star(i):
    indices, distances = tree.query_radius([coords_radians[i]], r=proximity_threshold, return_distance=True, sort_results=True)
    valid_distances = distances[0][distances[0] > min_distance]
    if len(valid_distances) > 0:
        nearest_distance = valid_distances[0] * 180 / np.pi
        nearest_star_indices = indices[0][distances[0] > min_distance]
        if len(nearest_star_indices) > 0:
            nearest_star_index = nearest_star_indices[0]
            return (i, ra_list[i], dec_list[i], nearest_star_index, ra_list[nearest_star_index], dec_list[nearest_star_index], nearest_distance)
    return (i, ra_list[i], dec_list[i], None, None, None, np.inf)



def estimate_execution_time(n_samples):
    # Tomamos una muestra de n_samples estrellas para medir el tiempo
    sample_indices = np.random.choice(n_stars, n_samples, replace=False)

    start_time = time.time()

    # Usamos ProcessPoolExecutor para paralelizar
    with ProcessPoolExecutor() as executor:
        temp = list(executor.map(find_nearest_star, sample_indices))
        results_df = pd.DataFrame(temp, columns=[
        'Star_Index', 'Star_RA', 'Star_DEC', 
        'Nearest_Star_Index', 'Nearest_Star_RA', 'Nearest_Star_DEC', 
        'Distance_Degrees'
    ])
    # Guardar los resultados en un archivo CSV
    results_df.to_csv('distancias_estrellas_test.csv', index=False)
    elapsed_time = time.time() - start_time

    # Estimamos el tiempo total para todas las estrellas
    time_per_star = elapsed_time / n_samples
    total_estimated_time = time_per_star * n_stars

    print(f"Tiempo promedio por estrella: {time_per_star:.4f} segundos")
    print(f"Tiempo estimado total para {n_stars} estrellas: {total_estimated_time / 60:.2f} minutos")
    return total_estimated_time


if __name__ == '__main__':
    n_samples = 800  # Número de estrellas de muestra
    estimate_execution_time(n_samples)
    #find_nearest_star()


#com="""
if __name__ == '__main__':
    subdivisions = 1000
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(find_nearest_star, range(n_stars)))

    # Crear DataFrame con los resultados
    results_df = pd.DataFrame(results, columns=[
        'Star_Index', 'Star_RA', 'Star_DEC', 
        'Nearest_Star_Index', 'Nearest_Star_RA', 'Nearest_Star_DEC', 
        'Distance_Degrees'
    ])
    results_df.to_csv('distancias_estrellas.csv', index=False)
#"""



# Assuming find_nearest_star is already defined and works on a batch of stars
com ="""
def process_batch(batch_indices):
    batch_results = []
    for i in batch_indices:
        result = find_nearest_star(i)  # Existing function that processes one star index
        batch_results.append(result)
    return batch_results

if __name__ == '__main__':
    subdivisions = 5000  # Number of stars per subset (batch size)
    
    # Split indices into batches
    def split_indices_into_batches(n_stars, batch_size):
        return [range(i, min(i + batch_size, n_stars)) for i in range(0, n_stars, batch_size)]
    
    # Split the range of star indices into batches
    batches = split_indices_into_batches(n_stars, subdivisions)
    
    all_results = []
    
    # Process each batch in parallel
    with ProcessPoolExecutor() as executor:
        # executor.map applies the function process_batch to each batch of star indices
        for batch_results in executor.map(process_batch, batches):
            all_results.extend(batch_results)  # Collect the results

            # Save incrementally after processing each batch
            temp_df = pd.DataFrame(batch_results, columns=[
                'Star_Index', 'Star_RA', 'Star_DEC', 
                'Nearest_Star_Index', 'Nearest_Star_RA', 'Nearest_Star_DEC', 
                'Distance_Degrees'
            ])
            temp_df.to_csv('distancias_estrellas.csv', mode='a', header=not all_results, index=False)  # Append results to the CSV
"""


Tiempo promedio por estrella: 0.0255 segundos
Tiempo estimado total para 40107889 estrellas: 17074.30 minutos


PROBANDO CON GPU --No funciona

In [23]:
tree_radius = proximity_threshold

def compute_distances_gpu(coords_radians, tree_radius, min_distance, results):
    idx = cuda.grid(1)
    print(f"{idx}")
    if idx < coords_radians.shape[0]:
        # Example calculation; replace with actual distance computation
        coord = coords_radians[idx]
        distance = np.sqrt(np.sum((coords_radians - coord)**2, axis=1))
        # Apply radius and min_distance filtering
        distances_within_radius = distance[distance < tree_radius]
        valid_distances = distances_within_radius[distances_within_radius > min_distance]
        if valid_distances.size > 0:
            nearest_distance = valid_distances[0]
            results[idx] = nearest_distance
        else:
            results[idx] = np.inf

def find_nearest_star(i, coords_radians, tree_radius, min_distance):
    # Allocate GPU memory
    d_coords_radians = cuda.to_device(coords_radians)
    d_results = cuda.device_array(coords_radians.shape[0], dtype=np.float32)
    
    # Launch kernel
    threads_per_block = 256
    blocks_per_grid = (coords_radians.shape[0] + (threads_per_block - 1)) // threads_per_block
    compute_distances_gpu[blocks_per_grid, threads_per_block](d_coords_radians, tree_radius, min_distance, d_results)
    
    # Copy results back to host
    results = d_results.copy_to_host()
    nearest_distance = results[i] if results[i] != np.inf else None
    return (i, nearest_distance)

if __name__ == '__main__':
    # Prepare data (example data here)
    with ProcessPoolExecutor() as executor:
        print("Running GPU routine...")
        # Lambda function to pass the parameters to find_nearest_star
        executor.map(find_nearest_star, range(len(coords_radians)))
    
    # Convert results to DataFrame and save
    results_df = pd.DataFrame(results_list, columns=['Star_Index', 'Nearest_Distance'])
    results_df.to_csv('distancias_estrellas.csv', index=False)

Running GPU routine...


KeyboardInterrupt: 

In [17]:
def find_nearest_star(i):
    current_star = coords[i]
    nearest_star_index = None
    nearest_distance = np.inf
    
    # Procesamos por bloques
    for start_idx in range(0, n_stars, block_size):
        end_idx = min(start_idx + block_size, n_stars)
        block_coords = coords[start_idx:end_idx]
        
        # Calculamos las separaciones del bloque actual
        separations = current_star.separation(block_coords).deg
        
        # Filtramos las estrellas que están más allá de la distancia mínima
        valid_indices = np.where(separations > min_distance)[0]
        
        if len(valid_indices) > 0:
            # Encontramos la más cercana en este bloque
            closest_in_block = valid_indices[np.argmin(separations[valid_indices])]
            closest_distance_in_block = separations[closest_in_block]
            
            # Actualizamos si es la más cercana encontrada
            if closest_distance_in_block < nearest_distance:
                nearest_distance = closest_distance_in_block
                nearest_star_index = start_idx + closest_in_block

    # Retornamos el resultado de esta estrella
    if nearest_star_index is not None:
        return f"Para la estrella {i}, la más cercana a partir de {min_distance} grados es la estrella {nearest_star_index} con una distancia de {nearest_distance} grados."
    else:
        return f"Para la estrella {i}, no se encontraron estrellas más cercanas que estén a más de {min_distance} grados."

# Paso 6: Paralelizar el proceso
if __name__ == '__main__':
    # Usamos ProcessPoolExecutor para ejecutar en paralelo
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(find_nearest_star, range(n_stars)))

    # Imprimimos los resultados
    #for result in results:
    #    print(result)

KeyboardInterrupt: 